Our merging document for the tables 

In [1]:
#First we have to load all necessary libraries

import pandas as pd
import numpy as np
import psycopg2 
import requests 
from zipfile import *
from sql_functions import get_engine
import sqlalchemy


In [2]:
import sql_functions as sf
from sql_functions import get_dataframe
schema = 'roundnet_pre'



After loading the necessary libraries we have to load the tables. As we have tables, there we want to add only one column from we want add them first before merging everything into one table. So we start with loading the surface into the tournament table

In [3]:
surface1= f'select * FROM {schema}.surface'
surface = get_dataframe(surface1)
surface.head()


,surface_id,name
0,1,Grass
1,2,Sand
2,3,Kunstrasen
3,4,Indoor
4,5,Snow


In [4]:
tournament1= f'select * FROM {schema}.tournament'
tournament_pre = get_dataframe(tournament1)
tournament_pre.head()

,event_id,event_name,date,location,surface_id,latitude,longitude,rating_tournament
0,8,Kiel Roundnet Open,2020-01-19,Kiel,4.0,54.322708,10.135555,0
1,29,Roundnet World Championships 2022,2022-09-08,Molenheide,1.0,51.082587,5.399360,0
2,102,EURA Grand Slam Freiburg,2022-07-30,Freiburg,1.0,47.990880,7.827049,0
3,2,Siegerland Open 2019,2019-11-16,Burbach,4.0,50.745600,8.080300,0
4,45,1. Roundnet Open Dresden,2020-08-30,Dresden,2.0,51.050409,13.737262,0


In [5]:
tournament=tournament_pre.merge(surface, how='left', on='surface_id') 

In [6]:
tournament.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   event_id           96 non-null     int64  
 1   event_name         96 non-null     object 
 2   date               96 non-null     object 
 3   location           96 non-null     object 
 4   surface_id         90 non-null     float64
 5   latitude           92 non-null     float64
 6   longitude          92 non-null     float64
 7   rating_tournament  96 non-null     int64  
 8   name               90 non-null     object 
dtypes: float64(3), int64(2), object(4)
memory usage: 7.5+ KB


In [7]:
tournament.head()

,event_id,event_name,date,location,surface_id,latitude,longitude,rating_tournament,name
0,8,Kiel Roundnet Open,2020-01-19,Kiel,4.0,54.322708,10.135555,0,Indoor
1,29,Roundnet World Championships 2022,2022-09-08,Molenheide,1.0,51.082587,5.399360,0,Grass
2,102,EURA Grand Slam Freiburg,2022-07-30,Freiburg,1.0,47.990880,7.827049,0,Grass
3,2,Siegerland Open 2019,2019-11-16,Burbach,4.0,50.745600,8.080300,0,Indoor
4,45,1. Roundnet Open Dresden,2020-08-30,Dresden,2.0,51.050409,13.737262,0,Sand


In [8]:
tournament.rename(columns={'name': 'surface'}, inplace=True)

In [9]:
tournament.rename(columns={'longitude': 'tournament_longitude'}, inplace=True)

In [10]:
tournament.rename(columns={'latitude': 'tournament_latitude'}, inplace=True)

In [11]:
tournament.drop('surface_id', inplace=True, axis=1)

As we decided to make a smaller tournament table, we need to do the same steps for tournament_small


In [12]:
tournament_small1= f'select * FROM {schema}.tournament_small'
tournament_spre = get_dataframe(tournament_small1)
tournament_spre.head()

,event_id,event_name,date,location,surface_id,latitude,longitude,rating_tournament
0,53,Deutsche Roundnet Meisterschaften 2021,2021-08-07,Münster,1,51.960665,7.626135,1
1,65,Lübecker Litschi Cup,2021-07-03,Lübeck,1,53.853832,10.683562,1
2,74,Roundnet Summerclash 2021,2021-09-18,Kassel,3,51.292938,9.485823,1
3,69,Roundnet Masters Leipzig,2021-07-31,Leipzig,1,51.340758,12.332942,1
4,90,Munich Indoor Shorty,2021-11-20,München,4,48.146662,11.498987,1


In [13]:
tournament_small=tournament_spre.merge(surface, how='left', on='surface_id')

In [14]:
tournament_small.rename(columns={'name': 'surface'}, inplace=True)
tournament_small.rename(columns={'longitude': 'tournament_longitude'}, inplace=True)
tournament_small.rename(columns={'latitude': 'tournament_latitude'}, inplace=True)
tournament_small.drop('surface_id', inplace=True, axis=1)

Following adding country to club


In [15]:
country1= f'select * FROM {schema}.country'
country = get_dataframe(country1)
country.head()

,country_id,name
0,1,Germany
1,2,Belgium
2,3,Netherlands
3,4,Czech Republic
4,5,Poland


In [16]:
club1= f'select * FROM {schema}.club'
club_pre = get_dataframe(club1)
club_pre.head()

,club_id,club_name,founding_date,country_id,club_insta,city,latitude,longitude,rg_member
0,1,1. Roundnetclub Opladen,2018-10-05,1,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1
1,2,1. Roundnet Club Köln,2016-09-09,1,1.roundnetclubkoeln,Köln,50.935909,6.875135,1
2,3,Roundnet Raccoons,2018-02-01,1,roundnet_raccoons,Kassel,51.315455,9.492410,0
3,4,Roundnet 069,2018-10-14,1,roundnet069,Frankfurt,50.110644,8.682092,0
4,5,Roundnet Gießen,None,1,roundnet_giessen,Gießen,50.586207,8.674231,1


In [17]:
club_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   club_id        92 non-null     int64  
 1   club_name      92 non-null     object 
 2   founding_date  39 non-null     object 
 3   country_id     92 non-null     int64  
 4   club_insta     75 non-null     object 
 5   city           92 non-null     object 
 6   latitude       92 non-null     float64
 7   longitude      92 non-null     float64
 8   rg_member      92 non-null     int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 6.6+ KB


In [18]:
club=club_pre.merge(country, how='left', on='country_id') 

In [19]:
club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   club_id        92 non-null     int64  
 1   club_name      92 non-null     object 
 2   founding_date  39 non-null     object 
 3   country_id     92 non-null     int64  
 4   club_insta     75 non-null     object 
 5   city           92 non-null     object 
 6   latitude       92 non-null     float64
 7   longitude      92 non-null     float64
 8   rg_member      92 non-null     int64  
 9   name           92 non-null     object 
dtypes: float64(2), int64(3), object(5)
memory usage: 7.9+ KB


In [20]:
club.rename(columns={'name': 'country'}, inplace=True)

In [21]:
club.rename(columns={'latitude': 'club_latitude'}, inplace=True)

In [22]:
club.rename(columns={'longitude': 'club_longitude'}, inplace=True)

In [23]:
club.drop('country_id', inplace=True, axis=1)

In [24]:
club.head()

,club_id,club_name,founding_date,club_insta,city,club_latitude,club_longitude,rg_member,country
0,1,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1,Germany
1,2,1. Roundnet Club Köln,2016-09-09,1.roundnetclubkoeln,Köln,50.935909,6.875135,1,Germany
2,3,Roundnet Raccoons,2018-02-01,roundnet_raccoons,Kassel,51.315455,9.492410,0,Germany
3,4,Roundnet 069,2018-10-14,roundnet069,Frankfurt,50.110644,8.682092,0,Germany
4,5,Roundnet Gießen,None,roundnet_giessen,Gießen,50.586207,8.674231,1,Germany


Now adding the country also to the player table

In [25]:
player1= f'select * FROM {schema}.player'
player_pre = get_dataframe(player1)
player_pre.head()

,player_id,first_name,last_name,player_city,player_eusra_id,club_id,country_id,gender,active_since,birth_date,player_insta,player_rg_elo,player_elo_games_total,rg_member,elo_division
0,2,Martin,Lang,Leverkusen,GERML99,1.0,1.0,m,2019.0,None,martingudelaune,1142.0,187,1,3.0
1,4,Sebastian,Köss,Köln,GERSK100,1.0,1.0,m,2019.0,1983-06-25,Herr_sybbi,1049.0,85,1,3.0
2,14,Jana,Esser,,None,1.0,1.0,f,2019.0,1991-11-29,None,951.0,10,1,2.0
3,64,Leonard,Breitzmann,,GERLB56,10.0,1.0,m,NaN,None,None,1106.0,39,1,3.0
4,65,Matias,Bermejo Pasetti,Basel,ESPMP5,10.0,24.0,m,2017.0,1994-04-27,wavingwave,1212.0,53,1,4.0


In [26]:
player_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_id               4115 non-null   int64  
 1   first_name              4115 non-null   object 
 2   last_name               4111 non-null   object 
 3   player_city             3179 non-null   object 
 4   player_eusra_id         297 non-null    object 
 5   club_id                 1941 non-null   float64
 6   country_id              4055 non-null   float64
 7   gender                  4112 non-null   object 
 8   active_since            510 non-null    float64
 9   birth_date              2106 non-null   object 
 10  player_insta            311 non-null    object 
 11  player_rg_elo           1504 non-null   float64
 12  player_elo_games_total  4115 non-null   int64  
 13  rg_member               4115 non-null   int64  
 14  elo_division            1504 non-null   

In [27]:
player=player_pre.merge(country, how='left', on='country_id') 

In [28]:
player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4115 entries, 0 to 4114
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_id               4115 non-null   int64  
 1   first_name              4115 non-null   object 
 2   last_name               4111 non-null   object 
 3   player_city             3179 non-null   object 
 4   player_eusra_id         297 non-null    object 
 5   club_id                 1941 non-null   float64
 6   country_id              4055 non-null   float64
 7   gender                  4112 non-null   object 
 8   active_since            510 non-null    float64
 9   birth_date              2106 non-null   object 
 10  player_insta            311 non-null    object 
 11  player_rg_elo           1504 non-null   float64
 12  player_elo_games_total  4115 non-null   int64  
 13  rg_member               4115 non-null   int64  
 14  elo_division            1504 non-null   

In [29]:
player.rename(columns={'name': 'country'}, inplace=True)

In [30]:
player.rename(columns={'gender': 'player_gender'}, inplace=True)

In [31]:
player.drop('country_id', inplace=True, axis=1)

In [32]:
player.head()

,player_id,first_name,last_name,player_city,player_eusra_id,club_id,player_gender,active_since,birth_date,player_insta,player_rg_elo,player_elo_games_total,rg_member,elo_division,country
0,2,Martin,Lang,Leverkusen,GERML99,1.0,m,2019.0,None,martingudelaune,1142.0,187,1,3.0,Germany
1,4,Sebastian,Köss,Köln,GERSK100,1.0,m,2019.0,1983-06-25,Herr_sybbi,1049.0,85,1,3.0,Germany
2,14,Jana,Esser,,None,1.0,f,2019.0,1991-11-29,None,951.0,10,1,2.0,Germany
3,64,Leonard,Breitzmann,,GERLB56,10.0,m,NaN,None,None,1106.0,39,1,3.0,Germany
4,65,Matias,Bermejo Pasetti,Basel,ESPMP5,10.0,m,2017.0,1994-04-27,wavingwave,1212.0,53,1,4.0,Spain


In [33]:
player['club_id'].convert_dtypes()

0          1
1          1
2          1
3         10
4         10
        ... 
4110    <NA>
4111    <NA>
4112    <NA>
4113    <NA>
4114      14
Name: club_id, Length: 4115, dtype: Int64

The second- last small table we want to add is the division_kat to division

In [34]:
division1= f'select * FROM {schema}.division'
division_pre = get_dataframe(division1)
division_pre.head()

,tournament_id,division_id,event_id,max_teams,starting_fee,starting_fee_reduced,livestream,use_lottery
0,3,4,2,16,,,None,0
1,4,2,2,16,,,None,0
2,5,4,3,64,,,None,0
3,6,3,3,32,,None,None,0
4,12,4,7,24,TBA,,None,0


In [35]:
division_kat1= f'select * FROM {schema}.division_kat'
division_kat= get_dataframe(division_kat1)
division_kat.head()

,division_id,name
0,1,Open
1,2,Mixed & Women
2,3,Women
3,4,Men
4,5,Mixed


In [36]:
division_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tournament_id         207 non-null    int64 
 1   division_id           207 non-null    int64 
 2   event_id              207 non-null    int64 
 3   max_teams             207 non-null    int64 
 4   starting_fee          207 non-null    object
 5   starting_fee_reduced  133 non-null    object
 6   livestream            34 non-null     object
 7   use_lottery           207 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 13.1+ KB


In [37]:
division=division_pre.merge(division_kat, how='left', on='division_id') 

In [38]:
division.head()

,tournament_id,division_id,event_id,max_teams,starting_fee,starting_fee_reduced,livestream,use_lottery,name
0,3,4,2,16,,,None,0,Men
1,4,2,2,16,,,None,0,Mixed & Women
2,5,4,3,64,,,None,0,Men
3,6,3,3,32,,None,None,0,Women
4,12,4,7,24,TBA,,None,0,Men


In [39]:
division.rename(columns={'name': 'division'}, inplace=True)

In [40]:
division.head()

,tournament_id,division_id,event_id,max_teams,starting_fee,starting_fee_reduced,livestream,use_lottery,division
0,3,4,2,16,,,None,0,Men
1,4,2,2,16,,,None,0,Mixed & Women
2,5,4,3,64,,,None,0,Men
3,6,3,3,32,,None,None,0,Women
4,12,4,7,24,TBA,,None,0,Men


In [41]:
division.drop('division_id', inplace=True, axis=1)

In [42]:
division.head()

,tournament_id,event_id,max_teams,starting_fee,starting_fee_reduced,livestream,use_lottery,division
0,3,2,16,,,None,0,Men
1,4,2,16,,,None,0,Mixed & Women
2,5,3,64,,,None,0,Men
3,6,3,32,,None,None,0,Women
4,12,7,24,TBA,,None,0,Men


the last small table we wanna add is team to tteam to get in our starting table for the big one (tteam) the gender in, which information we have in team

In [43]:
tteam1= f'select * FROM {schema}.tteam'
tteam_pre = get_dataframe(tteam1)
tteam_pre.head()

,tournament_team_id,team_name,position,team_id,tournament_id,team_skill,team_rg_id,elo_diff,quali_points
0,68,Blockwurst,1.0,58,3,1,NaN,NaN,NaN
1,69,Mzungu,2.0,59,3,1,NaN,NaN,NaN
2,70,The Rangers,3.0,60,3,1,NaN,NaN,NaN
3,71,Spiking Raccoons,4.0,61,3,1,NaN,NaN,NaN
4,72,PaderSpikers,5.0,62,3,1,NaN,NaN,NaN


In [44]:
team1= f'select * FROM {schema}.team'
team = get_dataframe(team1)
team.head()

,team_id,team_name,gender
0,58,Blockwurst,mm
1,59,Mzungu,mm
2,60,Big & Tasty,mm
3,61,Spiking Raccoons,mm
4,62,Paderoona,mm


In [45]:
team.rename(columns={'gender': 'team_gender'}, inplace=True)

In [46]:
tteam_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tournament_team_id  5515 non-null   int64  
 1   team_name           5280 non-null   object 
 2   position            4812 non-null   float64
 3   team_id             5515 non-null   int64  
 4   tournament_id       5515 non-null   int64  
 5   team_skill          5515 non-null   int64  
 6   team_rg_id          3099 non-null   float64
 7   elo_diff            3099 non-null   float64
 8   quali_points        154 non-null    float64
dtypes: float64(4), int64(4), object(1)
memory usage: 387.9+ KB


In [47]:
tteam=tteam_pre.merge(team, how='left', on=['team_id','team_name']) 

In [48]:
tteam.head(10)

,tournament_team_id,team_name,position,team_id,tournament_id,team_skill,team_rg_id,elo_diff,quali_points,team_gender
0,68,Blockwurst,1.0,58,3,1,NaN,NaN,NaN,mm
1,69,Mzungu,2.0,59,3,1,NaN,NaN,NaN,mm
2,70,The Rangers,3.0,60,3,1,NaN,NaN,NaN,NaN
3,71,Spiking Raccoons,4.0,61,3,1,NaN,NaN,NaN,mm
4,72,PaderSpikers,5.0,62,3,1,NaN,NaN,NaN,NaN
5,73,Schlammbeiser,6.0,63,3,1,NaN,NaN,NaN,mm
6,74,Bike'n'Spike,7.0,64,3,1,NaN,NaN,NaN,mm
7,75,Lang & Elend,8.0,65,3,1,NaN,NaN,NaN,mm
8,76,Team Erzquell,9.0,66,3,1,NaN,NaN,NaN,mm
9,77,Hau drauf!,10.0,67,3,1,NaN,NaN,NaN,mm


Now its time to try to generate our big one dataset

First we merge tournament on devision 

In [49]:
tournament_division=division.merge(tournament_small, how='left', on='event_id') 

In [50]:
tournament_division.head()

,tournament_id,event_id,max_teams,starting_fee,starting_fee_reduced,livestream,use_lottery,division,event_name,date,location,tournament_latitude,tournament_longitude,rating_tournament,surface
0,3,2,16,,,None,0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2,16,,,None,0,Mixed & Women,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,3,64,,,None,0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,3,32,,None,None,0,Women,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12,7,24,TBA,,None,0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
tournament_division.drop('event_id', inplace=True, axis=1)

Next we connect the tournament_division to the tteam

In [52]:
tournament_tteam=tteam.merge(tournament_division, how='left', on='tournament_id')

In [53]:
tournament_tteam.drop('tournament_id', inplace=True, axis=1)

In [54]:
tournament_tteam.head()

,tournament_team_id,team_name,position,team_id,team_skill,team_rg_id,elo_diff,quali_points,team_gender,max_teams,...,livestream,use_lottery,division,event_name,date,location,tournament_latitude,tournament_longitude,rating_tournament,surface
0,68,Blockwurst,1.0,58,1,NaN,NaN,NaN,mm,16.0,...,None,0.0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,69,Mzungu,2.0,59,1,NaN,NaN,NaN,mm,16.0,...,None,0.0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70,The Rangers,3.0,60,1,NaN,NaN,NaN,NaN,16.0,...,None,0.0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71,Spiking Raccoons,4.0,61,1,NaN,NaN,NaN,mm,16.0,...,None,0.0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72,PaderSpikers,5.0,62,1,NaN,NaN,NaN,NaN,16.0,...,None,0.0,Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
tournament_tteam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5515 entries, 0 to 5514
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tournament_team_id    5515 non-null   int64  
 1   team_name             5280 non-null   object 
 2   position              4812 non-null   float64
 3   team_id               5515 non-null   int64  
 4   team_skill            5515 non-null   int64  
 5   team_rg_id            3099 non-null   float64
 6   elo_diff              3099 non-null   float64
 7   quali_points          154 non-null    float64
 8   team_gender           4289 non-null   object 
 9   max_teams             5161 non-null   float64
 10  starting_fee          5161 non-null   object 
 11  starting_fee_reduced  3223 non-null   object 
 12  livestream            943 non-null    object 
 13  use_lottery           5161 non-null   float64
 14  division              5161 non-null   object 
 15  event_name           

Now we have to create the player tables properly. As the have the clubs information only for players: we merge it first to players: from the first try of the merging we see, that we need to do some adjustment:

In [56]:
player.rename(columns={'country': 'player_country'}, inplace=True)

In [57]:
club.rename(columns={'country': 'club_country'}, inplace=True)

In [58]:
club.rename(columns={'rg_member': 'club_rg_member'}, inplace=True)

In [59]:
player.rename(columns={'rg_member': 'player_rg_member'}, inplace=True)

In [60]:
player_club=player.merge(club, how='left', on='club_id')

In [61]:
player_club.drop('club_id', inplace=True, axis=1)

In [62]:
player_club.head()

,player_id,first_name,last_name,player_city,player_eusra_id,player_gender,active_since,birth_date,player_insta,player_rg_elo,...,elo_division,player_country,club_name,founding_date,club_insta,city,club_latitude,club_longitude,club_rg_member,club_country
0,2,Martin,Lang,Leverkusen,GERML99,m,2019.0,None,martingudelaune,1142.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
1,4,Sebastian,Köss,Köln,GERSK100,m,2019.0,1983-06-25,Herr_sybbi,1049.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
2,14,Jana,Esser,,None,f,2019.0,1991-11-29,None,951.0,...,2.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
3,64,Leonard,Breitzmann,,GERLB56,m,NaN,None,None,1106.0,...,3.0,Germany,Aachen Lobsters,None,roundnet_aachen,Aachen,50.776351,6.083862,0.0,Germany
4,65,Matias,Bermejo Pasetti,Basel,ESPMP5,m,2017.0,1994-04-27,wavingwave,1212.0,...,4.0,Spain,Aachen Lobsters,None,roundnet_aachen,Aachen,50.776351,6.083862,0.0,Germany


In [63]:
player_club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4115 entries, 0 to 4114
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_id               4115 non-null   int64  
 1   first_name              4115 non-null   object 
 2   last_name               4111 non-null   object 
 3   player_city             3179 non-null   object 
 4   player_eusra_id         297 non-null    object 
 5   player_gender           4112 non-null   object 
 6   active_since            510 non-null    float64
 7   birth_date              2106 non-null   object 
 8   player_insta            311 non-null    object 
 9   player_rg_elo           1504 non-null   float64
 10  player_elo_games_total  4115 non-null   int64  
 11  player_rg_member        4115 non-null   int64  
 12  elo_division            1504 non-null   float64
 13  player_country          4055 non-null   object 
 14  club_name               1940 non-null   

Now we wanna ad the tournament results to it from the players itself. therefore we need to load the table and make some further adjustments

In [64]:
tplayer1= f'select * FROM {schema}.tplayer'
tplayer = get_dataframe(tplayer1)
tplayer.head()

,id,player_rg_elo,player_id,tournament_id,elo_diff,elo_games,tournament_team_id
0,2688,1038,977,105.0,24,7,1059.0
1,2689,1011,957,105.0,-4,7,1061.0
2,2690,1011,956,105.0,-4,7,1061.0
3,2691,1111,116,105.0,22,7,1062.0
4,2692,1050,921,105.0,22,7,1062.0


In [65]:
tplayer.rename(columns={'player_rg_elo': 'starting_player_rg_elo'}, inplace=True)

In [66]:
tplayer.rename(columns={'elo_diff': 'elo_won_lose'}, inplace=True)

In [67]:
tplayer.drop('tournament_id', inplace=True, axis=1)

In [68]:
tplayer.drop('id', inplace=True, axis=1)

In [69]:
tplayer.head()

,starting_player_rg_elo,player_id,elo_won_lose,elo_games,tournament_team_id
0,1038,977,24,7,1059.0
1,1011,957,-4,7,1061.0
2,1011,956,-4,7,1061.0
3,1111,116,22,7,1062.0
4,1050,921,22,7,1062.0


now merging the player to the ttplayer, but on a right join to not miss players who not play an elo game. 

In [70]:
tplayer_club=tplayer.merge(player_club, how='right', on='player_id')

In [71]:
tplayer_club.drop('player_id', inplace=True, axis=1)

In [72]:
tplayer_club.head()

,starting_player_rg_elo,elo_won_lose,elo_games,tournament_team_id,first_name,last_name,player_city,player_eusra_id,player_gender,active_since,...,elo_division,player_country,club_name,founding_date,club_insta,city,club_latitude,club_longitude,club_rg_member,club_country
0,1044.0,-6.0,9.0,1507.0,Martin,Lang,Leverkusen,GERML99,m,2019.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
1,1038.0,36.0,12.0,1748.0,Martin,Lang,Leverkusen,GERML99,m,2019.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
2,1074.0,-74.0,9.0,2045.0,Martin,Lang,Leverkusen,GERML99,m,2019.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
3,1031.0,21.0,11.0,1177.0,Martin,Lang,Leverkusen,GERML99,m,2019.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany
4,1052.0,-8.0,10.0,1243.0,Martin,Lang,Leverkusen,GERML99,m,2019.0,...,3.0,Germany,1. Roundnetclub Opladen,2018-10-05,1.roundnetclubopladen,Leverkusen,51.062152,7.014899,1.0,Germany


In [73]:
tplayer_club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7932 entries, 0 to 7931
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   starting_player_rg_elo  5321 non-null   float64
 1   elo_won_lose            5321 non-null   float64
 2   elo_games               5321 non-null   float64
 3   tournament_team_id      5170 non-null   float64
 4   first_name              7932 non-null   object 
 5   last_name               7928 non-null   object 
 6   player_city             6996 non-null   object 
 7   player_eusra_id         1849 non-null   object 
 8   player_gender           7929 non-null   object 
 9   active_since            2400 non-null   float64
 10  birth_date              4533 non-null   object 
 11  player_insta            1494 non-null   object 
 12  player_rg_elo           5321 non-null   float64
 13  player_elo_games_total  7932 non-null   int64  
 14  player_rg_member        7932 non-null   

In [74]:
tplayer_club.columns

Index(['starting_player_rg_elo', 'elo_won_lose', 'elo_games',
       'tournament_team_id', 'first_name', 'last_name', 'player_city',
       'player_eusra_id', 'player_gender', 'active_since', 'birth_date',
       'player_insta', 'player_rg_elo', 'player_elo_games_total',
       'player_rg_member', 'elo_division', 'player_country', 'club_name',
       'founding_date', 'club_insta', 'city', 'club_latitude',
       'club_longitude', 'club_rg_member', 'club_country'],
      dtype='object')

Last step is to merge the tplayer_club to the tournament_tteam table as final roundnet table

In [75]:
roundnet_complete_pre=tournament_tteam.merge(tplayer_club, how='left', on='tournament_team_id')

In [76]:
roundnet_complete_pre.head()

,tournament_team_id,team_name,position,team_id,team_skill,team_rg_id,elo_diff,quali_points,team_gender,max_teams,...,elo_division,player_country,club_name,founding_date,club_insta,city,club_latitude,club_longitude,club_rg_member,club_country
0,68,Blockwurst,1.0,58,1,NaN,NaN,NaN,mm,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,69,Mzungu,2.0,59,1,NaN,NaN,NaN,mm,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70,The Rangers,3.0,60,1,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71,Spiking Raccoons,4.0,61,1,NaN,NaN,NaN,mm,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72,PaderSpikers,5.0,62,1,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
roundnet_complete_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8100 entries, 0 to 8099
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tournament_team_id      8100 non-null   int64  
 1   team_name               7854 non-null   object 
 2   position                7391 non-null   float64
 3   team_id                 8100 non-null   int64  
 4   team_skill              8100 non-null   int64  
 5   team_rg_id              5684 non-null   float64
 6   elo_diff                5684 non-null   float64
 7   quali_points            308 non-null    float64
 8   team_gender             6547 non-null   object 
 9   max_teams               7746 non-null   float64
 10  starting_fee            7746 non-null   object 
 11  starting_fee_reduced    5617 non-null   object 
 12  livestream              1877 non-null   object 
 13  use_lottery             7746 non-null   float64
 14  division                7746 non-null   

In [78]:
roundnet_complete_pre.columns

Index(['tournament_team_id', 'team_name', 'position', 'team_id', 'team_skill',
       'team_rg_id', 'elo_diff', 'quali_points', 'team_gender', 'max_teams',
       'starting_fee', 'starting_fee_reduced', 'livestream', 'use_lottery',
       'division', 'event_name', 'date', 'location', 'tournament_latitude',
       'tournament_longitude', 'rating_tournament', 'surface',
       'starting_player_rg_elo', 'elo_won_lose', 'elo_games', 'first_name',
       'last_name', 'player_city', 'player_eusra_id', 'player_gender',
       'active_since', 'birth_date', 'player_insta', 'player_rg_elo',
       'player_elo_games_total', 'player_rg_member', 'elo_division',
       'player_country', 'club_name', 'founding_date', 'club_insta', 'city',
       'club_latitude', 'club_longitude', 'club_rg_member', 'club_country'],
      dtype='object')

Now we wanna push the file to SQL to have a better look on the table and to decide, which columns we want to drop as we have the same information in other tables. 

In [79]:
table_name ='roundnet_complete_pre'

from sql_functions import get_engine
engine = get_engine()


In [80]:
#if engine!=None:
    #try:
       # roundnet_complete.to_sql(name=table_name, 
                       # con=engine, 
                       # schema=schema, 
                       # if_exists='replace',  
                       # index=False, 
                       # chunksize=5000, 
                       # method='multi') 
       # print(f"The {table_name} table was imported successfully.")
    # Error handling
  #  except (Exception, psycopg2.DatabaseError) as error:
      #  print(error)
      #  engine = None
#else:
    #print('No engine')

After loading the table to SQL (to delete the rows we dont need to) we now load the new table (roundnet_complete) to drop the rows we don't need

In [81]:
roundnet_complete1= f'select * FROM {schema}.roundnet_complett'
roundnet_complete= get_dataframe(roundnet_complete1)
roundnet_complete.head()

,tournament_team_id,team_name,position,team_id,team_skill,team_rg_id,elo_diff,quali_points,team_gender,max_teams,...,elo_division,player_country,club_name,founding_date,club_insta,city,club_latitude,club_longitude,club_rg_member,club_country
0,866,RCW Schwarzfisch,3.0,646,3,2060.0,50.0,NaN,fm,16.0,...,4.0,Germany,Roundnet Litschis Lübeck,None,roundnet.luebeck,Lübeck,53.865467,10.686559,0.0,Germany
1,866,RCW Schwarzfisch,3.0,646,3,2060.0,50.0,NaN,fm,16.0,...,4.0,Germany,1. Roundnet Club Würzburg,2019-12-16,roundnetclub.wuerzburg,Würzburg,49.792450,9.932966,0.0,Germany
2,867,Paul und Sophie,15.0,649,2,1900.0,-66.0,NaN,mf,16.0,...,2.0,Germany,Roundnet Litschis Lübeck,None,roundnet.luebeck,Lübeck,53.865467,10.686559,0.0,Germany
3,867,Paul und Sophie,15.0,649,2,1900.0,-66.0,NaN,mf,16.0,...,2.0,Germany,Roundnet Litschis Lübeck,None,roundnet.luebeck,Lübeck,53.865467,10.686559,0.0,Germany
4,868,Büffelhüfte,18.0,650,1,1960.0,-30.0,NaN,None,20.0,...,2.0,Germany,Roundnet Litschis Lübeck,None,roundnet.luebeck,Lübeck,53.865467,10.686559,0.0,Germany


In [82]:
roundnet_ranking=roundnet_complete

In [83]:
roundnet_ranking.drop(['tournament_team_id', 'team_id','quali_points','starting_fee', 'starting_fee_reduced', 'livestream', 'use_lottery', 'rating_tournament', 'player_eusra_id', 'active_since', 'player_insta', 'player_rg_elo', 'player_elo_games_total', 'player_rg_member', 'founding_date', 'club_insta', 'club_rg_member'], inplace=True, axis=1)

In [84]:
roundnet_ranking.head()

,team_name,position,team_skill,team_rg_id,elo_diff,team_gender,max_teams,division,event_name,date,...,player_city,player_gender,birth_date,elo_division,player_country,club_name,city,club_latitude,club_longitude,club_country
0,RCW Schwarzfisch,3.0,3,2060.0,50.0,fm,16.0,Mixed,Lübecker Litschi Cup,2021-07-03,...,Lübeck,f,1998-07-24,4.0,Germany,Roundnet Litschis Lübeck,Lübeck,53.865467,10.686559,Germany
1,RCW Schwarzfisch,3.0,3,2060.0,50.0,fm,16.0,Mixed,Lübecker Litschi Cup,2021-07-03,...,Würzburg,m,1996-02-27,4.0,Germany,1. Roundnet Club Würzburg,Würzburg,49.792450,9.932966,Germany
2,Paul und Sophie,15.0,2,1900.0,-66.0,mf,16.0,Mixed,Lübecker Litschi Cup,2021-07-03,...,,f,2001-05-07,2.0,Germany,Roundnet Litschis Lübeck,Lübeck,53.865467,10.686559,Germany
3,Paul und Sophie,15.0,2,1900.0,-66.0,mf,16.0,Mixed,Lübecker Litschi Cup,2021-07-03,...,,m,2000-10-13,2.0,Germany,Roundnet Litschis Lübeck,Lübeck,53.865467,10.686559,Germany
4,Büffelhüfte,18.0,1,1960.0,-30.0,None,20.0,Men,Lübecker Litschi Cup,2021-07-03,...,Lübeck,m,1982-06-09,2.0,Germany,Roundnet Litschis Lübeck,Lübeck,53.865467,10.686559,Germany


In [85]:
roundnet_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5285 entries, 0 to 5284
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   team_name               5281 non-null   object 
 1   position                5273 non-null   float64
 2   team_skill              5285 non-null   int64  
 3   team_rg_id              5240 non-null   float64
 4   elo_diff                5240 non-null   float64
 5   team_gender             4666 non-null   object 
 6   max_teams               5285 non-null   float64
 7   division                5285 non-null   object 
 8   event_name              5285 non-null   object 
 9   date                    5285 non-null   object 
 10  location                5285 non-null   object 
 11  tournament_latitude     5285 non-null   float64
 12  tournament_longitude    5285 non-null   float64
 13  surface                 5285 non-null   object 
 14  starting_player_rg_elo  5170 non-null   

In [86]:
table_name ='roundnet_ranking'


from sql_functions import get_engine
engine = get_engine()

In [87]:
#if engine!=None:
    #try:
        #roundnet_ranking.to_sql(name=table_name, 
                       # con=engine, 
                       # schema='roundnet', 
                       # if_exists='replace', 
                       # index=False, 
                       # chunksize=5000, 
                       # method='multi')
      #  print(f"The {table_name} table was imported successfully.")
    # Error handling
  #  except (Exception, psycopg2.DatabaseError) as error:
      #  print(error)
      #  engine = None
#else:
  #  print('No engine')

We found out that we can get a table with all the tournament and not just the ranking ones. So this is how we did it

In [88]:
ranking_team_player1= f'select * FROM {schema}.ranking_team_player'
ranking_team_player = get_dataframe(ranking_team_player1)
ranking_team_player.head()

,id,player_id,team_id
0,76,245,58
1,77,65,58
2,78,66,59
3,79,67,59
4,80,68,60


In [89]:
first_help_table=ranking_team_player.merge(team, how='left', on='team_id')

In [90]:
second_help_table=first_help_table.merge(player_club, how='left', on='player_id')

In [91]:
tournament_division=division.merge(tournament, how='left', on='event_id')

In [92]:
tournament_tteam=tteam.merge(tournament_division, how='left', on='tournament_id')

In [93]:
tournament_roundnet_pre=tournament_tteam.merge(second_help_table, how='left', on=['team_id', 'team_name'])

In [94]:

tournament_roundnet_pre.info()
tournament_roundnet_pre.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10688 entries, 0 to 10687
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tournament_team_id      10688 non-null  int64  
 1   team_name               10453 non-null  object 
 2   position                9265 non-null   float64
 3   team_id                 10688 non-null  int64  
 4   tournament_id           10688 non-null  int64  
 5   team_skill              10688 non-null  int64  
 6   team_rg_id              5785 non-null   float64
 7   elo_diff                5785 non-null   float64
 8   quali_points            303 non-null    float64
 9   team_gender_x           9462 non-null   object 
 10  event_id                9598 non-null   float64
 11  max_teams               9598 non-null   float64
 12  starting_fee            9598 non-null   object 
 13  starting_fee_reduced    6383 non-null   object 
 14  livestream              1774 non-null 

,tournament_team_id,team_name,position,team_id,tournament_id,team_skill,team_rg_id,elo_diff,quali_points,team_gender_x,...,elo_division,player_country,club_name,founding_date,club_insta,city,club_latitude,club_longitude,club_rg_member,club_country
0,68,Blockwurst,1.0,58,3,1,NaN,NaN,NaN,mm,...,3.0,Germany,Aachen Lobsters,None,roundnet_aachen,Aachen,50.776351,6.083862,0.0,Germany
1,68,Blockwurst,1.0,58,3,1,NaN,NaN,NaN,mm,...,4.0,Spain,Aachen Lobsters,None,roundnet_aachen,Aachen,50.776351,6.083862,0.0,Germany
2,69,Mzungu,2.0,59,3,1,NaN,NaN,NaN,mm,...,NaN,Germany,Roundnet Heidelberg,None,roundnetheidelberg,Heidelberg,49.409358,8.694724,1.0,Germany
3,69,Mzungu,2.0,59,3,1,NaN,NaN,NaN,mm,...,4.0,Germany,Roundnet Siegerland,None,spikeballsiegerland,Burbach,50.745946,8.080256,0.0,Germany
4,70,The Rangers,3.0,60,3,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,71,Spiking Raccoons,4.0,61,3,1,NaN,NaN,NaN,mm,...,3.0,Germany,Roundnet Raccoons,2018-02-01,roundnet_raccoons,Kassel,51.315455,9.492410,0.0,Germany
6,71,Spiking Raccoons,4.0,61,3,1,NaN,NaN,NaN,mm,...,4.0,Germany,Roundnet Raccoons,2018-02-01,roundnet_raccoons,Kassel,51.315455,9.492410,0.0,Germany
7,72,PaderSpikers,5.0,62,3,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,73,Schlammbeiser,6.0,63,3,1,NaN,NaN,NaN,mm,...,2.0,Germany,Roundnet Gießen,None,roundnet_giessen,Gießen,50.586207,8.674231,1.0,Germany
9,73,Schlammbeiser,6.0,63,3,1,NaN,NaN,NaN,mm,...,3.0,Germany,Roundnet Gießen,None,roundnet_giessen,Gießen,50.586207,8.674231,1.0,Germany


In [95]:
tournament_roundnet_pre.drop(['tournament_team_id', 'team_id','quali_points','starting_fee', 'starting_fee_reduced', 'livestream', 'use_lottery', 'player_eusra_id', 'active_since', 'player_insta', 'player_rg_member', 'founding_date', 'club_insta', 'club_rg_member'], inplace=True, axis=1)

In [96]:
table_name='tournament_roundnet_pre'

In [97]:
#if engine!=None:
    #try:
        #tournament_roundnet_pre.to_sql(name=table_name, 
                       # con=engine, 
                       # schema='roundnet_pre', 
                        #if_exists='replace', 
                       # index=False, 
                      #  chunksize=5000, 
                      #  method='multi')
       # print(f"The {table_name} table was imported successfully.")
    # Error handling
  #  except (Exception, psycopg2.DatabaseError) as error:
      #  print(error)
      #  engine = None
#else:
   # print('No engine')

The tournament_roundnet_pre table was imported successfully.


After loading the file into dbeaver we want to look which tables we wanna drop in our final table